In [6]:
import pandas as pd
import ee
import geemap
import ipyleaflet # to show the map in the notebook

## Packages required for using Dynamic World in a Jupyter Notebook

`EarthEngine`: 

Installation Tutorial: https://developers.google.com/earth-engine/guides/python_install-conda#mac_6

This tutorial requires the installation of Google Cloud: using the following command:
    
```bash
curl https://sdk.cloud.google.com | bash
```

To authenticate, you should: `earthengine authenticate` as the ee.Authenticate() function is not working. (it only works in Google Colab Notebooks)


### How to authenticate? 
Instructions are here: https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup


### How to install the earthengine command-line tool in the Google Cloud Shell:
        
```bash
curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py && python get-pip.py --user
pip install earthengine-api --upgrade --user
echo 'export PATH="$HOME/.local/bin:$PATH"' >> ~/.bashrc
source ~/.bashrc
```

You can validate this installation by using `earthengine authenticate` in the Google Cloud Shell.
            
- `GeeMap`: 


In [7]:
ee.Initialize(project="jpmorgancapstone")

In [8]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
import ipyleaflet
m = ipyleaflet.Map()
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [10]:
)

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…